### Installing dependencies

<details>
<summary>📌 Cell Description: Installing Required Libraries</summary>

This cell prepares the full software environment needed for the research project. Since the study is based on Data Science and Astronomy, it depends on a collection of scientific Python libraries. This cell installs those libraries to ensure that all later code cells run smoothly without errors.

The installation includes machine-learning frameworks, data-processing tools, astronomy-specific libraries, and visualization packages. PyTorch is installed first because it is the main framework used for building and training models. Data-science libraries such as NumPy, Pandas, and Scikit-Learn are included to load, clean, and analyze astronomical datasets. Visualization tools like Matplotlib, Seaborn, and Plotly allow the research to generate clear graphs that help explain patterns in astronomical observations. Astropy is installed because it provides essential astronomy utilities such as coordinate handling and time conversions that normal data-science libraries cannot perform.

The cell also installs “captum,” which is used to interpret machine-learning models and understand why a model makes certain predictions — very important in scientific research where transparency is required. “tqdm” is added to provide progress bars during long computations, making the workflow more understandable and trackable.

Finally, NumPy and Pandas are force-reinstalled to avoid version conflicts, ensuring the environment remains stable. Running this cell is a necessary first step because every other part of the research — data loading, feature extraction, modelling, and interpretation — depends on these tools being available.

</details>


In [ ]:
# Install required packages for the project
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
%pip install scikit-learn pandas numpy matplotlib seaborn umap-learn
%pip install captum plotly astropy tqdm
"""
captum: For model interpretability
plotly: For interactive visualizations
astropy: For astronomical computations
tqdm: For progress bars
"""

%pip install --upgrade --force-reinstall numpy pandas
print("✅ All packages installed successfully!")

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 91.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 63.1 MB/s eta 0:00:0000:01:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 24.4 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: tzdata
    Found existing installation: tzdata 2025.2
    Uninstalling tzdata-2025.2:
      Successfully uninstalled tzdata-2025.2
  Attempting uninstall: six
    Found existing installatio

✅ All packages installed successfully!


<details>
<summary>📌 Cell Description: Searching & Downloading ZTF Astronomical Images</summary>

This cell creates the functions needed to communicate with an online astronomy data archive (IRSA) and fetch real astronomical images from the Zwicky Transient Facility (ZTF). This step is important because the research uses actual telescope data. The code connects securely using a username and password stored in environment variables, searches the database for images based on field number or sky coordinates, retrieves the matching results, and can then download a selected file to the local computer. This entire process automates data collection, making it easier to build datasets for analysis without manually downloading each file from the website.

The cell performs three main tasks:  
(1) loading login details stored safely in a `.env` file,  
(2) defining a function to search the ZTF database for images, and  
(3) defining another function to download specific images returned by the search.  
This ensures that the research workflow is smooth, repeatable, and scalable when handling hundreds or thousands of astronomical files.

---

### 🔹 **Key Points **

- **Secure Access Setup:**  
  The cell loads your username and password from a hidden `.env` file, keeping login information safe instead of writing it inside the code.

- **Connects to the ZTF Astronomy Database:**  
  It communicates with IRSA (NASA’s Infrared Science Archive), which hosts ZTF images of stars, galaxies, and transient astronomical events.

- **Flexible Search Function:**  
  A function (`search_ztf_images`) allows you to search for telescope images using:  
  - a **field number**, or  
  - **sky coordinates** (Right Ascension and Declination).  
  This helps you find exactly the region of the sky you want to study.

- **Automatically Reads Results into a Data Table:**  
  If the search succeeds, the results are converted into a Pandas DataFrame for easy viewing and filtering.  
  The researcher can immediately inspect metadata like observation time, filter used, and exposure ID.

- **Download Function for Telescope Images:**  
  The second function (`download_ztf_file`) can download an image file directly to your computer using the file path from the metadata table.

- **Error-Checking Included:**  
  The code prints clear messages if searching or downloading fails, which makes debugging easier.

- **Real Astronomical Data Retrieval:**  
  By running this cell, the researcher can automatically fetch real ZTF scientific images, which form the foundation of the upcoming analysis, feature extraction, and modelling stages.

- **Fully Reproducible:**  
  Anyone repeating the research can call these same functions to obtain the exact same astronomical images, which is critical for scientific correctness.

---

### ⭐ **Why This Cell Is Important for the Research**
This project depends on analyzing real astronomical observations. Manually downloading each image file would be inefficient and error-prone. This code makes the process automatic and reproducible. It ensures that the dataset used for machine learning and data analysis comes directly from a trusted scientific archive (IRSA), with no manual steps that could introduce mistakes.  
It also shows the researcher's ability to integrate external scientific data sources into a complete data science pipeline.

</details>


In [1]:
import os
import requests
import pandas as pd
from io import StringIO
from dotenv import load_dotenv

load_dotenv()

USER = os.getenv("IRSA_USER")
PASS = os.getenv("IRSA_PASS")


# Step 1: Search for available ZTF images
def search_ztf_images(field=None, ra=None, dec=None):
    """Search for ZTF images"""
    search_url = "https://irsa.ipac.caltech.edu/ibe/search/ztf/products/sci"

    params = {'ct': 'csv'}

    # Build WHERE clause based on search criteria
    where_clauses = []
    if field:
        where_clauses.append(f"field={field}")
    if ra and dec:
        where_clauses.append(f"POS={ra},{dec}")

    if where_clauses:
        params['WHERE'] = " AND ".join(where_clauses)

    response = requests.get(search_url, params=params, auth=(USER, PASS))

    if response.status_code == 200:
        # Parse CSV response
        df = pd.read_csv(StringIO(response.text))
        return df
    else:
        print(f"Search failed: {response.status_code}")
        print(response.text)
        return None

# Step 2: Download a specific file
def download_ztf_file(file_path, output_filename):
    """Download a ZTF file using the file path from search results"""
    base_url = "https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci"
    full_url = f"{base_url}/{file_path}"

    print(f"Downloading: {full_url}")

    response = requests.get(full_url, auth=(USER, PASS))

    if response.status_code == 200:
        with open(output_filename, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded: {output_filename}")
        return True
    else:
        print(f"Download failed: {response.status_code}")
        print(response.text[:200])
        return False

# Example usage
print("Searching for ZTF images...")
results = search_ztf_images(field=570)  

if results is not None and len(results) > 0:
    print(f"Found {len(results)} images")
    print("\nFirst few results:")
    print(results.head())

    # Download the first image
    if 'filefracday' in results.columns:
        first_file = results.iloc[0]
        # Construct the file path from the metadata
        # This depends on the actual column names in the results
        print("\nAttempting to download first image...")
        # You'll need to construct the path based on the actual column names
        print(first_file)
else:
    print("No results found")


Searching for ZTF images...
Found 83243 images

First few results:
           ra        dec  infobits  field  ccdid  qid  rcid  fid filtercode  \
0  141.623367  22.229255         0    570     16    2    61    3         zi   
1  141.621517  22.223860         0    570     16    2    61    3         zi   
2  141.596689  22.216610         0    570     16    2    61    2         zr   
3  141.618880  22.231302         0    570     16    2    61    1         zg   
4  141.608133  22.218416         0    570     16    2    61    2         zr   

             pid  ...         ra1       dec1         ra2       dec2  \
0  1773523966115  ...  142.080359  22.671668  141.144493  22.651365   
1  1769477366115  ...  142.078570  22.666203  141.142651  22.646049   
2  1081478866115  ...  142.053598  22.659099  141.117847  22.638809   
3  2488485436115  ...  142.076277  22.673442  141.140153  22.653875   
4  2825517046115  ...  142.065580  22.660484  141.129400  22.641115   

          ra3       dec3       